## Air Quality Dashboard
This notebook shows how to use IPython Widgets to display the air quality using Grove Sensors.

In [1]:
import collect_air_data as air_data
import time

In [2]:
pms5003_data = air_data.pms5003.read()
print(pms5003_data.pm_per_1l_air(0.3))
print(pms5003_data.pm_per_1l_air(0.5))
print(pms5003_data.pm_per_1l_air(1.0))
print(pms5003_data.pm_per_1l_air(2.5))
print(pms5003_data.pm_per_1l_air(5.0))
print(pms5003_data.pm_per_1l_air(10.0))

354
62
45
6
4
2


In [3]:
print(air_data.gas_gmxxxb.getGM102B_raw())
print(air_data.gas_gmxxxb.getGM302B_raw())
print(air_data.gas_gmxxxb.getGM502B_raw())
print(air_data.gas_gmxxxb.getGM702B_raw())

518
407
330
286


In [4]:
while True:
    if air_data.bme_680.get_sensor_data():
        output = '{0:.2f} C,{1:.2f} hPa,{2:.2f} %RH'.format(
            air_data.bme_680.data.temperature,
            air_data.bme_680.data.pressure,
            air_data.bme_680.data.humidity)
        #Only print gas data if the heater is stable
        if air_data.bme_680.data.heat_stable:
            print('{0},{1} Ohms'.format(
                output,
                air_data.bme_680.data.gas_resistance))
            break
        else:
            print(output)
    time.sleep(1)

22.36 C,1016.79 hPa,50.33 %RH
22.37 C,1016.81 hPa,50.36 %RH
22.38 C,1016.79 hPa,50.34 %RH
22.39 C,1016.80 hPa,50.30 %RH
22.40 C,1016.81 hPa,50.26 %RH,6737.862373984357 Ohms


In [ ]:
%matplotlib notebook
import matplotlib

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox
import random
import threading
from IPython.display import display, clear_output
import time
import math
from redpitaya.overlay.mercury import mercury as FPGA

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.animation import FuncAnimation
from datetime import datetime, date
import pandas as pd

overlay = FPGA()
ANALOG_TEMP = FPGA.analog_in(1) # define which pin will be used for temperature sensor

In [5]:
air_data.analog_temp()

23.00330906632371

In [12]:
air_data.analog_voc_ratio()

0.7997111689897652

In [13]:
air_data.analog_alcohol_ratio()

1.7804557189442372

In [14]:
air_data.reset_initial_values()

(0.1883111926978958, 2.5867110196956413)

In [15]:
air_data.voc_air_ref

0.14

In [ ]:
air_data.alcohol_air_ref

In [ ]:
ANALOG_Alcohol_pin = FPGA.analog_in(3) # define which pin will be used for alcohol sensor
Alcohol_air = ANALOG_Alcohol_pin.read()
print(Alcohol_air)

In [ ]:
ANALOG_Alcohol_pin = FPGA.analog_in(3) # define which pin will be used for alcohol sensor
Alcohol_volt = ANALOG_Alcohol_pin.read()
Alcohol_gas = Alcohol_volt/(5.0-Alcohol_volt)
#Scale the result to be between 2.0 and 30.0 to keep on a similar scale to temperature
Alcohol_ratio = (1/(Alcohol_gas/1.885))*2
print(Alcohol_volt,Alcohol_gas,Alcohol_ratio)

In [ ]:
df = pd.DataFrame() #Create an empty data frame to append to later
#timestamp = df.size
def animate(i):
    global df
    clean_data_table = pd.DataFrame() #Create an empty data frame to append to later
    latest_temp=temp()[0]
    voc_volt = voc_air_pin.read()
    voc_gas = voc_volt/(5.0)
    #Scale the result to be between 2.0 and 30.0 to keep on a similar scale to temperature
    voc_ratio = (voc_gas/voc_air)*28
    Alcohol_volt = ANALOG_Alcohol_pin.read()
    Alcohol_gas = Alcohol_volt/(5.0-Alcohol_volt)
    #Scale the result to be between 2.0 and 30.0 to keep on a similar scale to temperature
    Alcohol_ratio = (1/(Alcohol_gas/1.885))*1.5
    clear_output(wait=True)
    clean_data_table['Temperature (oC)'] = [latest_temp]
    clean_data_table['VOC in Air'] = [voc_ratio]
    clean_data_table['Alcohol in Air'] = [Alcohol_ratio]
#    clean_data_table['Date'] = [datetime.now()]
    timestamp = df.size
    timestamp = 2
    df = df.append(clean_data_table, ignore_index = True)
    df.index.name = 'timestamp'
    
    #x = df['Date']
    x = df.index
    y1 = df['Temperature (oC)']
    y2 = df['VOC in Air']
    y3 = df['Alcohol in Air']
    
    plt.cla()
    plt.plot(x,y1,label='Temperature (oC)')
    plt.plot(x,y2,label='VOC in Air')
    plt.plot(x,y3,label='Alcohol in Air')
    plt.xlabel('Timestamp')
    plt.ylabel('Temprature (C) / VOC')
    plt.title("Room Temperature & Air Quality")
    plt.gcf().autofmt_xdate()
    plt.ticklabel_format(useOffset=False,axis='y')
#    plt.legend()
    plt.show()
    
ani = FuncAnimation(plt.gcf(), animate,interval=2000)
plt.show()

In [ ]:
df.size

In [ ]:
df.index.name = 'timestamp'
df

In [ ]:
df.index.name = 'timestamp'
filename = "CSV-Data/Vicks " + str(datetime.now()) + ".csv"
df.to_csv(filename)

The CSV data files are then uploaded to [Edge Impulse](https://edgeimpulse.com) to make a machine learning model.